In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,models,losses,regularizers,constraints


In [12]:
tf.keras.backend.clear_session()

model = models.Sequential()
model.add(layers.Dense(64, input_dim = 64, kernel_regularizer = regularizers.l1_l2(0.01,0.01),
                      #kernel_regularizer = regularizers.l2(0.01),
                      #activity_regularizer = regularizers.l1(0.01),
                      kernel_constraint = constraints.MaxNorm(max_value = 2, axis = 0)))

model.add(layers.Dense(10,
                      kernel_regularizer = regularizers.l1_l2(0.01,0.01),activation = 'sigmoid' ))

model.compile(optimizer = 'rmsprop',
             loss = 'sparse_categorical_crossentropy', metrics = ['AUC'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 4,810
Trainable params: 4,810
Non-trainable params: 0
_________________________________________________________________


对于keras模型，目标函数中的正则化项一般在各层中指定，例如使用Dense的 kernel_regularizer 和 bias_regularizer等参数指定权重使用l1或者l2正则化项，此外还可以用kernel_constraint 和 bias_constraint等参数约束权重的取值范围，这也是一种正则化手段。

回归模型，loss function通常为 mean_squared_error。

二分类模型，loss function通常为 binary_crossentropy。

对于多分类模型，如果label是one-hot编码的，则使用交叉熵损失函数 categorical_crossentropy。
             如果label是序号编码的，则需要使用稀疏类别交叉熵损失函数 sparse_categorical_crossentropy。

常用的一些内置损失函数说明如下。

mean_squared_error（平方差误差损失，用于回归，简写为 mse, 类实现形式为 MeanSquaredError 和 MSE）

mean_absolute_error (绝对值误差损失，用于回归，简写为 mae, 类实现形式为 MeanAbsoluteError 和 MAE)

mean_absolute_percentage_error (平均百分比误差损失，用于回归，简写为 mape, 类实现形式为 MeanAbsolutePercentageError 和 MAPE)

Huber(Huber损失，只有类实现形式，用于回归，介于mse和mae之间，对异常值比较鲁棒，相对mse有一定的优势)

binary_crossentropy(二元交叉熵，用于二分类，类实现形式为 BinaryCrossentropy)

categorical_crossentropy(类别交叉熵，用于多分类，要求label为onehot编码，类实现形式为 CategoricalCrossentropy)

sparse_categorical_crossentropy(稀疏类别交叉熵，用于多分类，要求label为序号编码形式，类实现形式为 SparseCategoricalCrossentropy)

hinge(合页损失函数，用于二分类，最著名的应用是作为支持向量机SVM的损失函数，类实现形式为 Hinge)

kld(相对熵损失，也叫KL散度，常用于最大期望算法EM的损失函数，两个概率分布差异的一种信息度量。类实现形式为 KLDivergence 或 KLD)

cosine_similarity(余弦相似度，可用于多分类，类实现形式为 CosineSimilarity)

# 评估指标Metrics

MeanSquaredError（平方差误差，用于回归，可以简写为MSE，函数形式为mse）

MeanAbsoluteError (绝对值误差，用于回归，可以简写为MAE，函数形式为mae)

MeanAbsolutePercentageError (平均百分比误差，用于回归，可以简写为MAPE，函数形式为mape)

RootMeanSquaredError (均方根误差，用于回归)

Accuracy (准确率，用于分类，可以用字符串"Accuracy"表示，Accuracy=(TP+TN)/(TP+TN+FP+FN)，要求y_true和y_pred都为类别序号编码)

Precision (精确率，用于二分类，Precision = TP/(TP+FP))

Recall (召回率，用于二分类，Recall = TP/(TP+FN))

TruePositives (真正例，用于二分类)

TrueNegatives (真负例，用于二分类)

FalsePositives (假正例，用于二分类)

FalseNegatives (假负例，用于二分类)

AUC(ROC曲线(TPR vs FPR)下的面积，用于二分类，直观解释为随机抽取一个正样本和一个负样本，正样本的预测值大于负样本的概率)

CategoricalAccuracy（分类准确率，与Accuracy含义相同，要求y_true(label)为onehot编码形式）

SparseCategoricalAccuracy (稀疏分类准确率，与Accuracy含义相同，要求y_true(label)为序号编码形式)

MeanIoU (Intersection-Over-Union，常用于图像分割)

TopKCategoricalAccuracy (多分类TopK准确率，要求y_true(label)为onehot编码形式)

SparseTopKCategoricalAccuracy (稀疏多分类TopK准确率，要求y_true(label)为序号编码形式)

Mean (平均值)

Sum (求和)

# 优化器

深度学习优化算法大概经历了 SGD -> SGDM -> NAG ->Adagrad -> Adadelta(RMSprop) -> Adam -> Nadam 这样的发展历程。

对于一般新手炼丹师，优化器直接使用Adam，并使用其默认参数就OK了。

一些爱写论文的炼丹师由于追求评估指标效果，可能会偏爱前期使用Adam优化器快速下降，后期使用SGD并精调优化器参数得到更好的结果。

此外目前也有一些前沿的优化算法，据称效果比Adam更好，例如LazyAdam, Look-ahead, RAdam, Ranger等.

In [13]:
#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)
    

In [14]:
# 求f(x) = a*x**2 + b*x + c的最小值

# 使用optimizer.apply_gradients

x = tf.Variable(0.0, name = 'x', dtype = tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)

@tf.function
def minimizef():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    
    while tf.constant(True):
        with tf.GradientTape() as tape:
            y = a*tf.pow(x,2) + b*x +c
        dy_dx = tape.gradient(y,x)
        optimizer.apply_gradients(grads_and_vars = [(dy_dx,x)])
        
        #迭代终止条件
        if tf.abs(dy_dx) < tf.constant(0.00001):
            break
        
        if tf.math.mod(optimizer.iterations, 100) ==0:
            printbar()
            tf.print('step == ',optimizer.iterations)
            tf.print('x ==', x)
            tf.print('')
            
    y = a*tf.pow(x,2) + b*x + c
    return y

tf.print('y =', minimizef())
tf.print('x =', x)


================================================================================14:18:54
step ==  100
x == 0.867380381

================================================================================14:18:54
step ==  200
x == 0.98241204

================================================================================14:18:54
step ==  300
x == 0.997667611

================================================================================14:18:54
step ==  400
x == 0.999690711

================================================================================14:18:54
step ==  500
x == 0.999959

================================================================================14:18:54
step ==  600
x == 0.999994516

y = 0
x = 0.999995232


In [17]:
# 求f(x) = a*x**2 + b*x + c的最小值

# 使用optimizer.minimize

x = tf.Variable(0.0,name = "x",dtype = tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)   

def f():   
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x,2)+b*x+c
    return(y)

@tf.function
def train(epoch):  
    for _ in tf.range(epoch):  
        optimizer.minimize(f,[x])
    tf.print("epoch = ",optimizer.iterations)
    return(f())

train(1000)
tf.print("y = ",f())
tf.print("x = ",x)

epoch =  1000
y =  0
x =  0.999998569
